In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import plotly.express as px
import plotly.graph_objects as go
import json
from pathlib import Path
from pandas import json_normalize

In [2]:
import sys
sys.path.append("..")  # go up from notebooks/ to project root

## Import hyperliquid data 

In [3]:
path = Path.cwd().parent / "data" / "hyperliquid" / "SOL.json"

df = pd.read_json(path, lines=True)
raw_df = json_normalize(df["raw"])
df = pd.concat([df.drop(columns=["raw"]), raw_df], axis=1)

df.head()

,time,ver_num,channel,data.coin,data.time,data.levels
0,2023-09-16T09:00:01.039593170,1,l2Book,SOL,1694854799647,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'..."
1,2023-09-16T09:00:01.207203705,1,l2Book,SOL,1694854800124,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'..."
2,2023-09-16T09:00:01.346960349,1,l2Book,SOL,1694854800701,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'..."
3,2023-09-16T09:00:01.909415502,1,l2Book,SOL,1694854801653,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'..."
4,2023-09-16T09:00:02.510527925,1,l2Book,SOL,1694854802240,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'..."


In [4]:
mid_prices = []
for levels in df["data.levels"]: 
    bids = levels[0]
    asks = levels[1]
    best_bid = max(float(level["px"]) for level in bids)
    best_ask = min(float(level["px"]) for level in asks)

    mid = (best_bid + best_ask) * 0.5
    mid_prices.append(mid)

df["mid"] = mid_prices
df.head()


,time,ver_num,channel,data.coin,data.time,data.levels,mid
0,2023-09-16T09:00:01.039593170,1,l2Book,SOL,1694854799647,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'...",19.11
1,2023-09-16T09:00:01.207203705,1,l2Book,SOL,1694854800124,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'...",19.11
2,2023-09-16T09:00:01.346960349,1,l2Book,SOL,1694854800701,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'...",19.11
3,2023-09-16T09:00:01.909415502,1,l2Book,SOL,1694854801653,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'...",19.11
4,2023-09-16T09:00:02.510527925,1,l2Book,SOL,1694854802240,"[[{'px': '19.107', 'sz': '633.53', 'n': 2}, {'...",19.11


In [5]:
from src.backtest.engine import BacktestEngine, BacktestConfig
from src.backtest.strategy import BuyAndHoldStrategy

strategy = BuyAndHoldStrategy()
config = BacktestConfig(initial_capital=1_000.0)

engine = BacktestEngine(prices=df, strategy=strategy, config=config)
result = engine.run()

equity_curve = result.equity_curve
summary = result.summary

print(equity_curve.head())
print(summary)


     mid  target_weight  equity  cash  position_qty  position_value  returns
0  19.11            1.0  1000.0   0.0     52.328624          1000.0      0.0
1  19.11            1.0  1000.0   0.0     52.328624          1000.0      0.0
2  19.11            1.0  1000.0   0.0     52.328624          1000.0      0.0
3  19.11            1.0  1000.0   0.0     52.328624          1000.0      0.0
4  19.11            1.0  1000.0   0.0     52.328624          1000.0      0.0
{'initial_capital': 1000.0, 'final_equity': 999.0842490842492, 'total_pnl': -0.9157509157507775, 'total_return': -0.0009157509157508015, 'max_drawdown': -0.007592501105072924, 'strategy_name': 'BuyAndHold'}


In [6]:
# Plot equity curve using plotly 
fig = px.line(equity_curve, x=equity_curve.index, y="equity", title="Equity Curve")
fig.show()

In [8]:
from src.visualisation.orderbook import make_orderbook_histogram, make_orderbook
from src.data.hyperliquid import load_orderbook_and_mid_series

path = Path.cwd().parent / "data" / "hyperliquid" / "SOL.json"
mid_df, snapshots = load_orderbook_and_mid_series(path)

fig = make_orderbook_histogram(snapshots, top_n=10)
fig2 = make_orderbook(snapshots, top_n=5)
fig.show()
fig2.show()
